In [1]:
import psycopg2

In [17]:
def get_cursor():
    conn = psycopg2.connect(
    database="legislative_testimony",
    user="brianrennie",
    password="")
    cursor = conn.cursor()
    return cursor

cursor = get_cursor()

In [20]:
cursor.execute('''select action, 
date, LAG(DATE_PART('day', date)) over (Order by date) from bill_history''')

cursor.fetchall()

[('Rules suspended', datetime.datetime(2021, 1, 14, 5, 0), None),
 ('Referred to the temporary committee on House Rules',
  datetime.datetime(2021, 1, 14, 5, 0),
  14.0),
 ('Rules suspended', datetime.datetime(2021, 1, 14, 5, 0), 14.0),
 ('Committee recommended ought to pass',
  datetime.datetime(2021, 1, 14, 5, 0),
  14.0),
 ('Read third and passed to be engrossed',
  datetime.datetime(2021, 1, 14, 5, 0),
  14.0),
 ('Read second, amended and ordered to a third reading',
  datetime.datetime(2021, 1, 14, 5, 0),
  14.0),
 ('Placed on file', datetime.datetime(2021, 1, 19, 16, 13, 33, 657000), 14.0),
 ('Referred to the committee on Temporary Ways and Means',
  datetime.datetime(2021, 1, 19, 16, 13, 33, 657000),
  19.0),
 ('Referred to the committee on Temporary Senate Rules',
  datetime.datetime(2021, 1, 19, 16, 13, 33, 657000),
  19.0),
 ('Placed on file', datetime.datetime(2021, 1, 19, 16, 13, 33, 657000), 19.0),
 ('Read, rules suspended, read second and ordered to a third reading',
  da